In [1]:
import os
os.chdir("../")
%pwd

'd:\\A - My Projects\\A - MLOps\\Flower-Gift-Helper'

# Entity

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    folder_dir: Path
    source_URL: str
    data_file_path: Path
    unzip_folder_dir: Path
    data_dir: Path
    train_dir: Path
    test_dir: Path
    params_test_split: float

# Configuration Manager

In [3]:
from flowerClassifier import logger, CONFIG_FILE_PATH, PARAMS_FILE_PATH
from flowerClassifier.utils.common import read_yaml, create_directories

In [4]:
class ConfigManager():
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root_dir])
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        
        create_directories([config.folder_dir])
        
        data_ingestion_config = DataIngestionConfig(
            folder_dir=config.folder_dir,
            source_URL=config.source_URL,
            data_file_path=config.data_file_path,
            unzip_folder_dir=config.unzip_folder_dir,
            data_dir=config.data_dir,
            train_dir=config.train_dir,
            test_dir=config.test_dir,
            params_test_split=self.params.TEST_SPLIT,
        )
        
        return data_ingestion_config


# Component

In [5]:
import os
import zipfile
import gdown
from flowerClassifier import logger
from sklearn.model_selection import train_test_split
import shutil

In [8]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    
    def download_file(self)-> str:
        '''
        Fetch data from the url
        '''
        try: 
            dataset_url=self.config.source_URL
            zip_download_dir=self.config.data_file_path
            os.makedirs(self.config.folder_dir, exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id, zip_download_dir)
            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")
        except Exception as e:
            raise e
        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_folder_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.data_file_path, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
            
            
    def split_data(self):
        # Create train and validation directories if they don't exist
        os.makedirs(self.config.train_dir, exist_ok=True)
        os.makedirs(self.config.test_dir, exist_ok=True)

        # Get all class folders
        classes = os.listdir(self.config.data_dir)

        for cls in classes:
            # Create class folders in train and validation directories
            os.makedirs(os.path.join(self.config.train_dir, cls), exist_ok=True)
            os.makedirs(os.path.join(self.config.test_dir, cls), exist_ok=True)

            # Get all images in the class folder
            images = os.listdir(os.path.join(self.config.data_dir, cls))

            # Split the images into train and validation sets
            train_images, val_images = train_test_split(images, test_size=self.config.params_test_split, random_state=42)

            # Move train images
            for img in train_images:
                src = os.path.join(self.config.data_dir, cls, img)
                dst = os.path.join(self.config.train_dir, cls, img)
                shutil.copy(src, dst)

            # Move validation images
            for img in val_images:
                src = os.path.join(self.config.data_dir, cls, img)
                dst = os.path.join(self.config.test_dir, cls, img)
                shutil.copy(src, dst)
                
        logger.info(f"Data split successfully completed.")

# Pipeline

In [9]:
try:
    config = ConfigManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
    data_ingestion.split_data()
except Exception as e:
    raise e

[2024-09-07 09:14:47,182 | INFO] common: yaml file: constant\config.yaml loaded successfully
[2024-09-07 09:14:47,184 | INFO] common: yaml file: constant\params.yaml loaded successfully
[2024-09-07 09:14:47,185 | INFO] common: created directory at: artifacts
[2024-09-07 09:14:47,186 | INFO] common: created directory at: artifacts/data_ingestion
[2024-09-07 09:14:47,187 | INFO] 3879767291: Downloading data from https://drive.google.com/file/d/1fQyEq3wMn5hmonwp632xlGTTtDzVxFZz/view?usp=sharing into file artifacts/data_ingestion/data.zip


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1fQyEq3wMn5hmonwp632xlGTTtDzVxFZz
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1fQyEq3wMn5hmonwp632xlGTTtDzVxFZz&confirm=t&uuid=cb0df22f-7ba4-485a-b207-72d05f860e89
To: d:\A - My Projects\A - MLOps\Flower-Gift-Helper\artifacts\data_ingestion\data.zip
100%|██████████| 215M/215M [00:26<00:00, 8.12MB/s] 

[2024-09-07 09:15:18,166 | INFO] 3879767291: Downloaded data from https://drive.google.com/file/d/1fQyEq3wMn5hmonwp632xlGTTtDzVxFZz/view?usp=sharing into file artifacts/data_ingestion/data.zip


[2024-09-07 09:15:21,341 | INFO] 3879767291: Data split successfully completed.
